# Used Car Analysis

In [98]:
#Import needed modules and dataset(s) to be used
#Dataset acquired from: 
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn
import plotly
import plotly.express as px
import plotly.graph_objects as go
import datetime

pd.set_option('display.max.columns', None)
cars_data = pd.read_csv('cars_raw.csv')

In [99]:
#Take a first look at dataframe
cars_data.head(5)

,Year,Make,Model,Used/New,Price,ConsumerRating,ConsumerReviews,SellerType,SellerName,SellerRating,SellerReviews,StreetName,State,Zipcode,DealType,ComfortRating,InteriorDesignRating,PerformanceRating,ValueForMoneyRating,ExteriorStylingRating,ReliabilityRating,ExteriorColor,InteriorColor,Drivetrain,MinMPG,MaxMPG,FuelType,Transmission,Engine,VIN,Stock#,Mileage
0,2019,Toyota,Sienna SE,Used,"$39,998",4.6,45,Dealer,CarMax Murrieta - Now offering Curbside Pickup...,3.3,3,25560 Madison Ave Murrieta,CA,92562,Great,4.7,4.6,4.6,4.4,4.6,4.7,Red,Black,Front-wheel Drive,19,27,Gasoline,8-Speed Automatic,3.5L V6 24V PDI DOHC,5TDXZ3DC2KS015402,22998646,29403
1,2018,Ford,F-150 Lariat,Used,"$49,985",4.8,817,Dealer,Giant Chevrolet,4.8,131,1001 S Ben Maddox Way Visalia,CA,93292,Good,4.9,4.8,4.8,4.6,4.8,4.7,Shadow Black,Black,Four-wheel Drive,19,24,Gasoline,10-Speed Automatic,3.5L V6 24V PDI DOHC Twin Turbo,1FTEW1EG2JFD44217,22418A,32929
2,2017,RAM,1500 Laramie,Used,"$41,860",4.7,495,Dealer,Gill Auto Group Madera,4.6,249,1100 S Madera Ave Madera,CA,93637,Good,4.8,4.7,4.8,4.6,4.8,4.7,Granite Crystal Clearcoat Metallic,Black,Four-wheel Drive,15,21,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV,1C6RR7VT5HS842283,NG277871G,23173
3,2021,Honda,Accord Sport SE,Used,"$28,500",5.0,36,Dealer,AutoSavvy Las Vegas,4.6,284,2121 E Sahara Ave Las Vegas,NV,89104,NaN,4.9,5.0,4.9,5.0,5.0,5.0,Gray,–,Front-wheel Drive,29,35,Gasoline,Automatic CVT,1.5L I4 16V GDI DOHC Turbo,1HGCV1F49MA038035,54237,10598
4,2020,Lexus,RX 350,Used,"$49,000",4.8,76,Dealer,Lexus of Henderson,4.8,4755,7737 Eastgate Rd Henderson,NV,89011,Good,4.9,4.8,4.8,4.7,4.8,4.9,Eminent White Pearl,Birch,Front-wheel Drive,20,27,Gasoline,8-Speed Automatic,3.5L V6 24V PDI DOHC,2T2AZMAA8LC156270,HDT4181A,28137


In [100]:
#Further Explore dataset
print(cars_data.describe(), end = '\n --- \n')
print(cars_data.isna().sum(), end = '\n --- \n')
print(cars_data['DealType'].value_counts(), end = '\n --- \n')
print(cars_data['Make'].value_counts())
print(cars_data['State'].value_counts())
print(cars_data.info())

              Year  ConsumerRating  ConsumerReviews  SellerRating  \
count  9379.000000     9379.000000      9379.000000   9379.000000   
mean   2018.721719        4.702825       133.187014      4.412571   
std       2.221708        0.240795       154.985640      0.626258   
min    2001.000000        2.500000         1.000000      1.000000   
25%    2018.000000        4.700000        30.000000      4.300000   
50%    2019.000000        4.800000        75.000000      4.600000   
75%    2020.000000        4.800000       182.000000      4.800000   
max    2022.000000        5.000000       817.000000      5.000000   

       SellerReviews  ComfortRating  InteriorDesignRating  PerformanceRating  \
count    9379.000000    9379.000000           9379.000000        9379.000000   
mean      984.089988       4.771895              4.727391           4.696290   
std      1609.039864       0.217822              0.194391           0.253664   
min         1.000000       3.000000              2.800000 

In [101]:
#Find and deal with missing values
print(cars_data[cars_data['DealType'].isna()]['DealType'])
cars_data['DealType'].fillna('No Data Available', inplace = True)
print(cars_data['DealType'].value_counts())

3       NaN
14      NaN
17      NaN
21      NaN
22      NaN
       ... 
8913    NaN
9138    NaN
9171    NaN
9194    NaN
9305    NaN
Name: DealType, Length: 222, dtype: object
Good                 5524
Great                2408
Fair                 1225
No Data Available     222
Name: DealType, dtype: int64


In [102]:
#Clean/create new columns

#Drop $ ,(,), and 'Not Priced' to make price into an int column
price_na_list = cars_data[cars_data['Price'].eq('Not Priced')].index.tolist()
cars_data.drop(labels = price_na_list, inplace = True)
cars_data['Price'] = cars_data['Price'].str.replace('$', '', regex = False)
cars_data['Price'] = cars_data['Price'].str.replace(',', '', regex = False)
cars_data['Price'] = pd.to_numeric(cars_data['Price'])
print(cars_data.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9374 entries, 0 to 9378
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Year                   9374 non-null   int64  
 1   Make                   9374 non-null   object 
 2   Model                  9374 non-null   object 
 3   Used/New               9374 non-null   object 
 4   Price                  9374 non-null   int64  
 5   ConsumerRating         9374 non-null   float64
 6   ConsumerReviews        9374 non-null   int64  
 7   SellerType             9374 non-null   object 
 8   SellerName             9374 non-null   object 
 9   SellerRating           9374 non-null   float64
 10  SellerReviews          9374 non-null   int64  
 11  StreetName             9374 non-null   object 
 12  State                  9374 non-null   object 
 13  Zipcode                9374 non-null   object 
 14  DealType               9374 non-null   object 
 15  Comf